In [2]:
import tensorflow as tf; print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

2023-09-19 03:07:24.106469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 03:07:24.817039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

Num GPUs Available:  4


In [1]:
import sys
sys.path.append('./.local/lib/python3.8/site-packages')
sys.path.append('/usr/lib/python3/dist-packages')

from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold
#from keras.utils.training_utils import multi_gpu_model


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98
GPU = 2
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU}"

print(datetime.datetime.now())

2023-09-22 17:36:47.332135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 17:36:48.078672: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

2023-09-22 17:36:49.462090


# Input loading

## signal only

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'child'
NORM = 'z-norm'
GENDER = 'female'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (7022, 5000, 12)
x test shape: (1756, 5000, 12)


In [3]:
hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (3131, 5000, 12)
x test shape: (793, 5000, 12)


In [4]:
y_train = y_train / 20
y_test = y_test / 20

## signal + features

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')


SCALE_Y = 100 if ADULT == 'adult' else 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
x train shape: (13358, 5000, 12)
x test shape: (3387, 5000, 12)
train: 0.6191248297691345, test: 0.6184705495834351


In [ ]:
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
c_train = np.load(input_path+'c_train.npz')['arr_0']
c_test = np.load(input_path+'c_test.npz')['arr_0']

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

# Settings

In [9]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers_kernel(9~19)_age%20(sigmoid)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64, 128] # num of filters(kernel)
stride_opts = [1,2,3]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,19,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 8, 16, 32, 64]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-21 15:18:47.064394


## 1d-cnn

In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
        out = inp


        # VGC block
        out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)



        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')


        # 4-fold cv
        kfold = KFold(nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]


            # model 학습
            try:
                # learning scheduler
                def step_decay(epoch):
                    start = 1e-3
                    drop = 0.1
                    epochs_drop = 10
                    lr = start * (drop ** np.floor((epoch)/epochs_drop))
                    return lr
                lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
                
                weightcache = f"{odir}/weights_{fold}.hdf5"
                
                model.compile(loss='mean_absolute_error', optimizer=Adam(lr=lr_scheduler, weight_decay=0), metrics=['mean_squared_error'])
                hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                model.load_weights(weightcache)
                y_pred = model.predict(x_test).flatten()

                # MAE 계산
                model_err = metrics.MeanAbsoluteError()
                model_err.update_state(y_test, y_pred)
                mae_val = model_err.result().numpy()
                test_mae.append(mae_val)

                # RMSE 계산
                model_err = metrics.RootMeanSquaredError() 
                model_err.update_state(y_test, y_pred)
                rmse_val = model_err.result().numpy()
                test_rmse.append(rmse_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # 

    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*20:.2f}+-{std_mae*20:.2f}_rmse{mean_rmse*20:.2f}+-{std_rmse*20:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*20:.2f}+-{std_mae*20:.2f}_rmse{mean_rmse*20:.2f}+-{std_rmse*20:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500
Epoch 1/100
67/74 [==========================>...] - ETA: 0s - loss: 0.1266 - mean_squared_error: 0.0323
Epoch 1: val_loss improved from inf to 0.10229, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_0.hdf5
74/74 [==============================] - 2s 12ms/step - loss: 0.1252 - mean_squared_error: 0.0314 - val_loss: 0.1023 - val_mean_squared_error: 0.0188
Epoch 2/100
73/74 [============================>.] - ETA: 0s - loss: 0.0822 - mean_squared_error: 0.0136
Epoch 2: val_loss did not improve from 0.10229
74/74 [==============================] - 1s 9ms/step - loss: 0.0824 - mean_squared_error: 0.0137 - val_loss: 0.2890 - val_mean_squared_error: 0.1256
Epoch 3/100
72/74 [============================>.] - ETA: 0s - loss: 0.0706 - mean_squared_error: 0.0103
Epoch 3: val_loss 

Epoch 1/100
71/74 [===========================>..] - ETA: 0s - loss: 0.1220 - mean_squared_error: 0.0303
Epoch 1: val_loss improved from inf to 0.10013, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_1.hdf5
74/74 [==============================] - 3s 16ms/step - loss: 0.1217 - mean_squared_error: 0.0300 - val_loss: 0.1001 - val_mean_squared_error: 0.0213
Epoch 2/100
72/74 [============================>.] - ETA: 0s - loss: 0.0823 - mean_squared_error: 0.0140
Epoch 2: val_loss did not improve from 0.10013
74/74 [==============================] - 1s 8ms/step - loss: 0.0822 - mean_squared_error: 0.0139 - val_loss: 0.1614 - val_mean_squared_error: 0.0467
Epoch 3/100
70/74 [===========================>..] - ETA: 0s - loss: 0.0666 - mean_squared_error: 0.0089
Epoch 3: val_loss improved from 0.1001

Epoch 1/100
71/74 [===========================>..] - ETA: 0s - loss: 0.1220 - mean_squared_error: 0.0317
Epoch 1: val_loss improved from inf to 0.26634, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_2.hdf5
74/74 [==============================] - 2s 12ms/step - loss: 0.1213 - mean_squared_error: 0.0313 - val_loss: 0.2663 - val_mean_squared_error: 0.1029
Epoch 2/100
69/74 [==========================>...] - ETA: 0s - loss: 0.0766 - mean_squared_error: 0.0116
Epoch 2: val_loss improved from 0.26634 to 0.11782, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_2.hdf5
74/74 [===========================

Epoch 1/100
74/74 [==============================] - ETA: 0s - loss: 0.1206 - mean_squared_error: 0.0320
Epoch 1: val_loss improved from inf to 0.12947, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_3.hdf5
74/74 [==============================] - 2s 13ms/step - loss: 0.1206 - mean_squared_error: 0.0320 - val_loss: 0.1295 - val_mean_squared_error: 0.0325
Epoch 2/100
72/74 [============================>.] - ETA: 0s - loss: 0.0783 - mean_squared_error: 0.0119
Epoch 2: val_loss improved from 0.12947 to 0.10281, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt9str3,layer2:c64filt13str1,layer3:c64filt17str3,1conv,dropout0.2,dnodes0,dropout0.4/weights_3.hdf5
74/74 [===========================

Epoch 1/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1116 - mean_squared_error: 0.0228
Epoch 1: val_loss improved from inf to 0.25155, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch128,layer1:c128filt17str3,layer2:c16filt11str3,layer3:c128filt13str1,layer4:c32filt11str2,1conv,dropout0,dnodes32,dropout0/weights_0.hdf5
19/19 [==============================] - 3s 45ms/step - loss: 0.1114 - mean_squared_error: 0.0227 - val_loss: 0.2515 - val_mean_squared_error: 0.1251
Epoch 2/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1017 - mean_squared_error: 0.0193
Epoch 2: val_loss did not improve from 0.25155
19/19 [==============================] - 0s 20ms/step - loss: 0.1016 - mean_squared_error: 0.0193 - val_loss: 0.3637 - val_mean_squared_error: 0.2098
Epoch 3/100
17/19 [=========================>....] - ETA: 0s - loss: 0.0843 - mean_squared_error: 0.0142
Epoch 3: val_lo

Epoch 1/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1140 - mean_squared_error: 0.0235
Epoch 1: val_loss improved from inf to 0.42985, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch128,layer1:c128filt17str3,layer2:c16filt11str3,layer3:c128filt13str1,layer4:c32filt11str2,1conv,dropout0,dnodes32,dropout0/weights_1.hdf5
19/19 [==============================] - 3s 52ms/step - loss: 0.1138 - mean_squared_error: 0.0235 - val_loss: 0.4298 - val_mean_squared_error: 0.2765
Epoch 2/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1013 - mean_squared_error: 0.0199
Epoch 2: val_loss improved from 0.42985 to 0.35698, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch128,layer1:c128filt17str3,layer2:c16filt11str3,layer3:c128filt13str1,layer4:c32filt11str2,1conv,dropout0,dnodes32,dropout0/weigh

Epoch 1/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1144 - mean_squared_error: 0.0241
Epoch 1: val_loss improved from inf to 0.19189, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch128,layer1:c128filt17str3,layer2:c16filt11str3,layer3:c128filt13str1,layer4:c32filt11str2,1conv,dropout0,dnodes32,dropout0/weights_2.hdf5
19/19 [==============================] - 3s 38ms/step - loss: 0.1144 - mean_squared_error: 0.0241 - val_loss: 0.1919 - val_mean_squared_error: 0.0790
Epoch 2/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1032 - mean_squared_error: 0.0203
Epoch 2: val_loss did not improve from 0.19189
19/19 [==============================] - 0s 20ms/step - loss: 0.1022 - mean_squared_error: 0.0200 - val_loss: 0.2717 - val_mean_squared_error: 0.1180
Epoch 3/100
17/19 [=========================>....] - ETA: 0s - loss: 0.0735 - mean_squared_error: 0.0105
Epoch 3: val_lo

Epoch 1/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1176 - mean_squared_error: 0.0251
Epoch 1: val_loss improved from inf to 0.13194, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch128,layer1:c128filt17str3,layer2:c16filt11str3,layer3:c128filt13str1,layer4:c32filt11str2,1conv,dropout0,dnodes32,dropout0/weights_3.hdf5
19/19 [==============================] - 2s 37ms/step - loss: 0.1165 - mean_squared_error: 0.0247 - val_loss: 0.1319 - val_mean_squared_error: 0.0413
Epoch 2/100
17/19 [=========================>....] - ETA: 0s - loss: 0.1096 - mean_squared_error: 0.0214
Epoch 2: val_loss did not improve from 0.13194
19/19 [==============================] - 0s 19ms/step - loss: 0.1096 - mean_squared_error: 0.0213 - val_loss: 0.2077 - val_mean_squared_error: 0.0967
Epoch 3/100
17/19 [=========================>....] - ETA: 0s - loss: 0.0930 - mean_squared_error: 0.0162
Epoch 3: val_lo

Epoch 1/100
74/74 [==============================] - ETA: 0s - loss: 0.1494 - mean_squared_error: 0.0507
Epoch 1: val_loss improved from inf to 0.11148, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt7str1,layer2:c16filt7str1,layer3:c64filt15str2,1conv,dropout0.1,dnodes16,dropout0.5/weights_0.hdf5
74/74 [==============================] - 3s 14ms/step - loss: 0.1494 - mean_squared_error: 0.0507 - val_loss: 0.1115 - val_mean_squared_error: 0.0221
Epoch 2/100
71/74 [===========================>..] - ETA: 0s - loss: 0.1060 - mean_squared_error: 0.0207
Epoch 2: val_loss improved from 0.11148 to 0.09215, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN_4layers_kernel(9~19)_age%20(sigmoid)_4fold_test500/batch32,layer1:c16filt7str1,layer2:c16filt7str1,layer3:c64filt15str2,1conv,dropout0.1,dnodes16,dropout0.5/weights_0.hdf5
74/74 [===========================

## cnn-transformer

In [ ]:
random.seed(98)

from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler


SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [5, 7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer_age%{SCALE_Y}(sigmoid)_loss(mae)-nodecay_{nfold}fold_{MAX_CASES}cases'
    odir_f = 'batch{}_cnn{}_filt{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    with tf.device("/CPU:0"):
        out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]))

        # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

        # conv 여러층    
        for i in range(clayer):
            out = Conv1D(filters=nfilt, kernel_size=filtsize, padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(poolsize, padding='same')(out)
        out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
        for i in range(tlayer):  # transformer
            attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
            attn_output = Dropout(droprate)(attn_output)
            out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
            ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
            out2 = Dropout(droprate)(ffn_output)
            out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
        out = GlobalMaxPooling1D()(out)

        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(fnode)(out)
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(1, activation='sigmoid')(out)

        if not os.path.exists(odir):
            os.mkdir(odir)


        model = Model(inputs=[inp], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]


        # model 학습
        try:
            # learning scheduler
            def step_decay(epoch):
                start = 1e-3
                drop = 0.1
                epochs_drop = 10
                lr = start * (drop ** np.floor((epoch)/epochs_drop))
                return lr
            lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(x_test).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue



    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128


2023-09-22 00:17:10.603307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 00:17:11.076687: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21956 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:c1:00.0, compute capability: 8.6


Epoch 1/100


2023-09-22 00:17:19.552360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 00:17:20.603293: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 00:17:20.679126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 00:17:20.773509: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x2f1d2c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 00:17:20.773544: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 00:17:20.779571: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 00:17:20.857025: I tensorf

42/42 [==============================] - ETA: 0s - loss: 0.0982 - mean_squared_error: 0.0214
Epoch 1: val_loss improved from inf to 0.08347, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
42/42 [==============================] - 27s 333ms/step - loss: 0.0982 - mean_squared_error: 0.0214 - val_loss: 0.0835 - val_mean_squared_error: 0.0094
Epoch 2/100
41/42 [============================>.] - ETA: 0s - loss: 0.0808 - mean_squared_error: 0.0125
Epoch 2: val_loss improved from 0.08347 to 0.07921, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
42/42 [==============================] - 13s 302ms/step - loss: 0.0807 - mean_squared_error: 0.0125 - val_lo

...

Epoch 1/100
41/42 [============================>.] - ETA: 0s - loss: 0.1068 - mean_squared_error: 0.0208
Epoch 1: val_loss improved from inf to 0.11002, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_0.hdf5
42/42 [==============================] - 12s 74ms/step - loss: 0.1069 - mean_squared_error: 0.0208 - val_loss: 0.1100 - val_mean_squared_error: 0.0217
Epoch 2/100
41/42 [============================>.] - ETA: 0s - loss: 0.0953 - mean_squared_error: 0.0174
Epoch 2: val_loss improved from 0.11002 to 0.08880, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_0.hdf5
42/42 [==============================] - 2s 43ms/step - loss: 0.0953 - mean_squared_error: 0.01

Epoch 1/100
41/42 [============================>.] - ETA: 0s - loss: 0.1076 - mean_squared_error: 0.0211
Epoch 1: val_loss improved from inf to 0.10696, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_1.hdf5
42/42 [==============================] - 13s 65ms/step - loss: 0.1076 - mean_squared_error: 0.0211 - val_loss: 0.1070 - val_mean_squared_error: 0.0208
Epoch 2/100
41/42 [============================>.] - ETA: 0s - loss: 0.1075 - mean_squared_error: 0.0210
Epoch 2: val_loss improved from 0.10696 to 0.10687, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_1.hdf5
42/42 [==============================] - 2s 49ms/step - loss: 0.1076 - mean_squared_error: 0.02

Epoch 1/100
41/42 [============================>.] - ETA: 0s - loss: 0.1165 - mean_squared_error: 0.0277
Epoch 1: val_loss improved from inf to 0.10450, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_2.hdf5
42/42 [==============================] - 13s 74ms/step - loss: 0.1165 - mean_squared_error: 0.0277 - val_loss: 0.1045 - val_mean_squared_error: 0.0201
Epoch 2/100
41/42 [============================>.] - ETA: 0s - loss: 0.0992 - mean_squared_error: 0.0186
Epoch 2: val_loss did not improve from 0.10450
42/42 [==============================] - 2s 40ms/step - loss: 0.0990 - mean_squared_error: 0.0185 - val_loss: 0.5594 - val_mean_squared_error: 0.3222
Epoch 3/100
41/42 [============================>.] - ETA: 0s - loss: 0.0830 - mean_squared_error: 0.0131
Epoch 3: val_loss did not improve from 0.10450
42/42 [=============

Epoch 1/100
41/42 [============================>.] - ETA: 0s - loss: 0.1082 - mean_squared_error: 0.0212
Epoch 1: val_loss improved from inf to 0.10639, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_3.hdf5
42/42 [==============================] - 13s 74ms/step - loss: 0.1082 - mean_squared_error: 0.0212 - val_loss: 0.1064 - val_mean_squared_error: 0.0208
Epoch 2/100
41/42 [============================>.] - ETA: 0s - loss: 0.1079 - mean_squared_error: 0.0210
Epoch 2: val_loss improved from 0.10639 to 0.10628, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size15_pool5_do0.1_tra3_head8_kdim128_fnn32/weights_3.hdf5
42/42 [==============================] - 2s 45ms/step - loss: 0.1081 - mean_squared_error: 0.02

Epoch 1/100
21/21 [==============================] - ETA: 0s - loss: 0.0890 - mean_squared_error: 0.0161
Epoch 1: val_loss improved from inf to 0.07768, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch256_cnn4_filt16_size5_pool2_do0.1_tra5_head8_kdim16_fnn32/weights_0.hdf5
21/21 [==============================] - 19s 300ms/step - loss: 0.0890 - mean_squared_error: 0.0161 - val_loss: 0.0777 - val_mean_squared_error: 0.0105
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 0.0794 - mean_squared_error: 0.0121
Epoch 2: val_loss improved from 0.07768 to 0.07607, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_female_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch256_cnn4_filt16_size5_pool2_do0.1_tra5_head8_kdim16_fnn32/weights_0.hdf5
21/21 [==============================] - 6s 266ms/step - loss: 0.0794 - mean_squared_error: 0.0121

## cnn-transformer + features

In [ ]:
from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# learning scheduler
def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

random.seed(98)
SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64, 128],
    "stride": [2,4,5],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, stride, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer+feats_age%{SCALE_Y}(sigmoid)_loss(mae)_{nfold}fold'
    odir_f = 'batch{}_cnn{}_filt{}_stride{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, stride, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    #strategy = tf.distribute.MirroredStrategy()
    #with strategy.scope():
    with tf.device(f'/gpu:{GPU}'):
        out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]), name='input_1')

        # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

        # conv 여러층    
        for i in range(clayer):
            out = Conv1D(filters=nfilt, kernel_size=filtsize, strides=stride, padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(poolsize, padding='same')(out)
        out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
        for i in range(tlayer):  # transformer
            attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
            attn_output = Dropout(droprate)(attn_output)
            out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
            ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
            out2 = Dropout(droprate)(ffn_output)
            out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
        out = GlobalMaxPooling1D()(out)

        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])
        
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(fnode)(out)
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(1, activation='sigmoid')(out)

        if not os.path.exists(odir):
            os.mkdir(odir)

        #cache_path = odir + "/initial_weights.hdf5"
        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]
        
        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        # model 학습
        try:
            # loading dataset
            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
                
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)



            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch256_cnn4_filt16_stride4_size11_pool5_do0.2_tra5_head2_kdim256_fnn128


2023-09-22 17:39:23.683901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 17:39:24.703052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:c1:00.0, compute capability: 8.6


Epoch 1/100


2023-09-22 17:39:37.511919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 17:39:37.988165: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 17:39:38.059190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 17:39:38.073051: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f698ff045e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 17:39:38.073091: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 17:39:38.081588: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 17:39:38.158901: I te

40/40 [==============================] - ETA: 0s - loss: 0.3784 - mean_squared_error: 0.1691
Epoch 1: val_loss improved from inf to 0.38420, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch256_cnn4_filt16_stride4_size11_pool5_do0.2_tra5_head2_kdim256_fnn128/weights_0.hdf5
40/40 [==============================] - 21s 87ms/step - loss: 0.3784 - mean_squared_error: 0.1691 - val_loss: 0.3842 - val_mean_squared_error: 0.1728
Epoch 2/100
39/40 [============================>.] - ETA: 0s - loss: 0.3800 - mean_squared_error: 0.1698
Epoch 2: val_loss did not improve from 0.38420
40/40 [==============================] - 2s 55ms/step - loss: 0.3799 - mean_squared_error: 0.1698 - val_loss: 0.3842 - val_mean_squared_error: 0.1728
Epoch 3/100
40/40 [==============================] - ETA: 0s - loss: 0.3799 - mean_squared_error: 0.1698
Epoch 3: val_loss did not improve from 0.38420
40/40 [============================

...

Epoch 1/100
157/157 [==============================] - ETA: 0s - loss: 0.3136 - mean_squared_error: 0.1305
Epoch 1: val_loss improved from inf to 0.16130, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn3_filt16_stride2_size7_pool2_do0.5_tra5_head8_kdim128_fnn64/weights_0.hdf5
157/157 [==============================] - 24s 55ms/step - loss: 0.3136 - mean_squared_error: 0.1305 - val_loss: 0.1613 - val_mean_squared_error: 0.0408
Epoch 2/100
157/157 [==============================] - ETA: 0s - loss: 0.2068 - mean_squared_error: 0.0657
Epoch 2: val_loss improved from 0.16130 to 0.15383, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn3_filt16_stride2_size7_pool2_do0.5_tra5_head8_kdim128_fnn64/weights_0.hdf5
157/157 [==============================] - 8s 49ms/step - loss: 0.2068 - mean_squared_error: 0.0657 -

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.2626 - mean_squared_error: 0.1005
Epoch 1: val_loss improved from inf to 0.11582, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch128_cnn3_filt16_stride5_size7_pool2_do0.5_tra4_head8_kdim16_fnn64/weights_0.hdf5
79/79 [==============================] - 16s 55ms/step - loss: 0.2626 - mean_squared_error: 0.1005 - val_loss: 0.1158 - val_mean_squared_error: 0.0221
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 0.2104 - mean_squared_error: 0.0686
Epoch 2: val_loss improved from 0.11582 to 0.10623, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch128_cnn3_filt16_stride5_size7_pool2_do0.5_tra4_head8_kdim16_fnn64/weights_0.hdf5
79/79 [==============================] - 3s 41ms/step - loss: 0.2104 - mean_squared_error: 0.0686 - val_los

Epoch 1/100
157/157 [==============================] - ETA: 0s - loss: 0.1320 - mean_squared_error: 0.0296
Epoch 1: val_loss improved from inf to 0.10649, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn2_filt16_stride2_size9_pool2_do0.1_tra5_head2_kdim64_fnn128/weights_0.hdf5
157/157 [==============================] - 21s 55ms/step - loss: 0.1320 - mean_squared_error: 0.0296 - val_loss: 0.1065 - val_mean_squared_error: 0.0177
Epoch 2/100
157/157 [==============================] - ETA: 0s - loss: 0.1103 - mean_squared_error: 0.0201
Epoch 2: val_loss improved from 0.10649 to 0.09931, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn2_filt16_stride2_size9_pool2_do0.1_tra5_head2_kdim64_fnn128/weights_0.hdf5
157/157 [==============================] - 8s 48ms/step - loss: 0.1103 - mean_squared_error: 0.0201 -

Epoch 1/100
157/157 [==============================] - ETA: 0s - loss: 0.1608 - mean_squared_error: 0.0432
Epoch 1: val_loss improved from inf to 0.10230, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn2_filt32_stride2_size11_pool5_do0.2_tra5_head8_kdim16_fnn64/weights_0.hdf5
157/157 [==============================] - 20s 46ms/step - loss: 0.1608 - mean_squared_error: 0.0432 - val_loss: 0.1023 - val_mean_squared_error: 0.0176
Epoch 2/100
157/157 [==============================] - ETA: 0s - loss: 0.1240 - mean_squared_error: 0.0252
Epoch 2: val_loss improved from 0.10230 to 0.09817, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch64_cnn2_filt32_stride2_size11_pool5_do0.2_tra5_head8_kdim16_fnn64/weights_0.hdf5
157/157 [==============================] - 6s 39ms/step - loss: 0.1240 - mean_squared_error: 0.0252 -

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.1882 - mean_squared_error: 0.0562
Epoch 1: val_loss improved from inf to 0.10341, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch128_cnn3_filt64_stride4_size9_pool4_do0.2_tra3_head2_kdim64_fnn32/weights_0.hdf5
79/79 [==============================] - 13s 50ms/step - loss: 0.1882 - mean_squared_error: 0.0562 - val_loss: 0.1034 - val_mean_squared_error: 0.0177
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 0.1409 - mean_squared_error: 0.0323
Epoch 2: val_loss improved from 0.10341 to 0.10257, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae)_4fold/batch128_cnn3_filt64_stride4_size9_pool4_do0.2_tra3_head2_kdim64_fnn32/weights_0.hdf5
79/79 [==============================] - 3s 40ms/step - loss: 0.1409 - mean_squared_error: 0.0323 - val_los